# TB2J optimization

This notebook contains prototype vectorized routines for TB2J. They focus on computing the $A^{uv}_{ij}$ tensor and the real-space Green's function.

In [1]:
import numpy as np
from itertools import product
from TB2J.exchange import ExchangeNCL
from HamiltonIO.siesta import SislParser
from TB2J.pauli import pauli_block_all

def vec_pauli_block(array):

    A00 = array[..., ::2, ::2]
    A01 = array[..., ::2, 1::2]
    A10 = array[..., 1::2, ::2]
    A11 = array[..., 1::2, 1::2]
    n2 = array.shape[-1] // 2

    out_dtype = np.result_type(array.dtype, np.complex64)
    block = np.empty((*array.shape[:-2], 4, n2, n2), dtype=out_dtype)

    np.add(A00, A11, out=block[..., 0, :, :]); block[..., 0, :, :] *= 0.5
    np.add(A01, A10, out=block[..., 1, :, :]); block[..., 1, :, :] *= 0.5
    np.subtract(A01, A10, out=block[..., 2, :, :]); block[..., 2, :, :] *= 0.5j
    np.subtract(A00, A11, out=block[..., 3, :, :]); block[..., 3, :, :] *= 0.5

    return block

def vec_pauli_sigma_norm(array):

    block = vec_pauli_block(array)[..., 1:, :, :]
    E = np.trace(block, axis1=-2, axis2=-1)
    E /= np.linalg.norm(E, axis=-1, keepdims=True)
    np.multiply(block, E[..., None, None], out=block)

    return block.sum(axis=-3)

def compute_GR(exchangencl, Gk):

    Rvecs = np.array(exchangencl.short_Rlist)
    kpts = exchangencl.G.kpts
    phase = np.exp(exchangencl.G.k2Rfactor * np.einsum('ni,mi->nm', Rvecs, kpts))
    phase *= exchangencl.G.kweights[None]
    GR = np.einsum('kij,rk->rij', Gk, phase, optimize='optimal')

    return GR

def compute_A_all(exchangencl, energy):

    Gk = exchangencl.G.get_Gk_all(energy)
    GR = compute_GR(exchangencl, Gk)

    magnetic_sites = exchangencl.ind_mag_atoms
    iorbs = [exchangencl.iorb(site) for site in magnetic_sites]
    P = [vec_pauli_sigma_norm(
        np.take(np.take(exchangencl.HR0, idx, axis=-2), idx, axis=-1)
    )
        for idx in iorbs
    ]

    A = {}
    for i, j in product(range(len(magnetic_sites)), repeat=2):
        idx, jdx = iorbs[i], iorbs[j]
        Gij = GR[:, idx][:, :, jdx]
        Gji = GR[:, jdx][:, :, idx]
        Gij = vec_pauli_block(Gij)
        Gji = vec_pauli_block(Gji)
        Gji = np.flip(Gji, axis=0)
        Pi = P[i]
        Pj = P[j]
        X = Pi @ Gij
        Y = Pj @ Gji
        mi, mj = (magnetic_sites[i], magnetic_sites[j])
        A[mi, mj] = np.einsum('...uij,...vji->...uv', X, Y) / np.pi

    return A

We vectorize about interaction vectors, kpoints, and $A^{uv}_{ij}$ components. It reduces the number of function calls and gets rid of unnecessary loops.

Let's see how these routines compare to the master version of TB2J. To do this, we select a moderate kmesh of $7 \times 7 \times 7$.

In [2]:
parser = SislParser(
    fdf_fname='siesta.fdf', ispin=None, read_H_soc=False, orth=False
)
model = parser.get_model()
basis = dict(zip(model.orbs, list(range(model.nbasis))))
exargs = dict(
    magnetic_elements=['Co'],
    include_orbs=None,
    kmesh=[7, 7, 7],
    emin=-12.0,
    emax=0.0,
    nz=100,
    exclude_orbs=[],
    Rcut=None,
    ne=None,
    nproc=1,
    use_cache=False,
    output_path="TB2J_results",
    orb_decomposition=False,
    orth=False,
    ibz=False,
    description="",
)
exchangencl = ExchangeNCL(
    tbmodels=model,
    atoms=model.atoms,
    basis=basis,
    efermi=0.0,
    **exargs
)
energies = exchangencl.contour.path

A gap is found at -22.473318471821468, set emin to it.
Magnetic atoms: [0, 1]


We now compare the performance of both approaches. First, the $A^{uv}_{ij}$ tensor can be calculated with TB2J as

In [10]:
from time import time

start_time = time()
A_dict = exchangencl.get_quantities_per_e(energies[50])
end_time = time()
regular_time1 = end_time - start_time
print(f"TB2J took {regular_time1:.3f} seconds to complete.")

TB2J took 28.392 seconds to complete.


With the vectorized routines the $A^{uv}_{ij}$ tensor is calculated as

In [12]:
start_time = time()
A_vec_dict = compute_A_all(exchangencl, energies[50])
end_time = time()
regular_time2 = end_time - start_time
print(f"Vectorized TB2J took {regular_time2:.3f} seconds to complete. A {(regular_time1 - regular_time2) / regular_time1 * 100:.2f}% time reduction!")

Vectorized TB2J took 3.454 seconds to complete. A 87.84% time reduction!


The last step is do a sanity check to make sure that the vectorized routines give the same results as before. 

In [17]:
for pair, Atensor in A_vec_dict.items():
    assert all(np.allclose(Atensor[i], A_dict['AijR'][Rm, *pair]) for i, Rm in enumerate(exchangencl.short_Rlist))
print('No assertion errors found, meaning that we obtain the same tensor as before.')

No assertion errors found, meaning that we obtain the same tensor as before.
